In [1]:
import lib as lib
import basis_set as bs
import numpy as np
import os
import mc_integration as mc

### Hartree Fock computation of the Helium atom

First we set up the simulation

In [2]:
N_electrons = 2
N_basis = 4
integrals_file = "integrals_He.npy"

normalized_wf = False

max_iter_SCF = 100
eps_SCF = 1E-5

Now we make minor modifications on the integral_master class to adapt it to the computation of the Helium atom, whose basis functions are different.

In [3]:
class integral_master_He():
    """
    Calculates, stores and retrieves the values of the <pr|g|qs> integrals
    """
    def __init__(self, dimension):
        """
        Initialization of the object

        Parameters
        ----------
        dimension : int
            Number of single basis functions

        Returns
        -------
        None
        """

        self.integral_dict_1 = None
        self.integral_dict_2 = None
        self.dimension = dimension

        return

    def calculate(self, file_name):
        """
        Calculates the <pr|g|qs> integrals and stores them in file

        Parameters
        ----------
        file_name : str
            File name that stores the values of the integrals

        Returns
        -------
        None
        """

        if file_name in os.listdir():
            print("Integral file already exists. Not computing the integrals. ")
            self.load_integrals(file_name)
            return

        integral_dict_1 = {}
        integral_dict_2 = {}

        # 1-body integrals
        for p in range(1, self.dimension + 1):
            for q in range(1, self.dimension + 1):
                if p == q:
                    I = self.calculate_1(p, q)
                else:
                    I = 0

                integral_dict_1[(p, q)] = I

        # 2-body integrals
        for p in range(1, self.dimension + 1):
            for q in range(1, p + 1):
                for r in range(1, p):
                    for s in range(1, r + 1):
                        I = self.calculate_2(p, r, q, s)

                        integral_dict_2[(p, r, q, s)] = I
                        integral_dict_2[(q, r, p, s)] = I
                        integral_dict_2[(p, s, q, r)] = I
                        integral_dict_2[(r, p, s, q)] = I
                        integral_dict_2[(q, s, p, r)] = I
                        integral_dict_2[(r, q, s, p)] = I
                        integral_dict_2[(s, p, r, q)] = I
                        integral_dict_2[(s, q, r, p)] = I
                r = p
                for s in range(1, q + 1):
                    I = self.calculate_2(p, r, q, s)

                    integral_dict_2[(p, r, q, s)] = I
                    integral_dict_2[(q, r, p, s)] = I
                    integral_dict_2[(p, s, q, r)] = I
                    integral_dict_2[(r, p, s, q)] = I
                    integral_dict_2[(q, s, p, r)] = I
                    integral_dict_2[(r, q, s, p)] = I
                    integral_dict_2[(s, p, r, q)] = I
                    integral_dict_2[(s, q, r, p)] = I

        np.save(file_name, np.array([integral_dict_1, integral_dict_2]))

        self.load_integrals(file_name)

        return


    def load_integrals(self, file_name):
        """
        Loads the values of the integrals in this object

        Parameters
        ----------
        file_name : str
            File name that stores the values of the integrals

        Returns
        -------
        None
        """

        self.integral_dict_1, self.integral_dict_2 = np.load(file_name, allow_pickle = True)

        return

    def get_1(self, p, q):
        """
        Returns the value of the h_pq integrals from the class dictionaries

        Parameters
        ----------
        p, q: int
            Indices that specify the h_pq integral

        Returns
        -------
        I : float
            Value of the h_pq integral
        """

        I = self.integral_dict_1[(p, q)]

        return I

    def get_2(self, p, r, q, s):
        """
        Returns the value of the <pr|g|qs> integrals from the class dictionaries

        Parameters
        ----------
        p, r, q, s: int
            Indeces that specify the <pr|g|qs> integral

        Returns
        -------
        I : float
            Value of the <pr|g|qs> integral
        """

        I = self.integral_dict_2[(p, r, q, s)]

        return I


    
    def calculate_1(self, p, q):
        """
        Calculates the value of the h_pq integrals by direct integration

        Parameters
        ----------
        p, q: int
            Indices that specify the h_pq integral

        Returns
        ----------
        I : float
            Value of the h_pq integral
        """

        I = 3*bs.ALPHA[p-1]*bs.ALPHA[q-1]*np.pi**(1.5)/(bs.ALPHA[p-1] + bs.ALPHA[q-1])**(5/2) - 4*np.pi/(bs.ALPHA[p-1] + bs.ALPHA[q-1])

        return I

    def calculate_2(self, p, r, q, s):
        """
        Calculates the value of the <pr|g|qs> integrals by monte carlo integration methods

        Parameters
        ----------
        p, q, r, s: int
            Indices that specify the <pr|g|qs> integral

        Returns
        ----------
        I : float
            Value of the <pr|g|qs> integral
        """

        system_size = 5
        N_walkers = 500
        N_steps = 12000
        N_skip = 2000
        trial_move = 0.6
        
        integrand = bs.He_two_body_integrand
        indices = np.array([p,r,q,s])
        dimension = 6

        sampling = bs.sampling_function

        I, deltaI, acceptance_ratio, trial_move = mc.MC_integration(sampling, integrand, indices, dimension, N_steps, N_walkers, N_skip, system_size, N_cores=-1, trial_move=trial_move)

        return I


### Computation block
In the next piece of code we run the actual computation of the energy levels of the Helium atom. First we initialize the class to compute the integrals using the four basis functions defined in Jos' book and then we initialize a random coeficient matrix and we compute the density matrix. From there we actually compute the matrix and then we check whether the wave functions are normalized and compute the matrix S accordingly.

In [4]:
integrals = integral_master_He(N_basis)
C = np.random.rand(N_basis, N_basis)
rho = lib.density_matrix(C, N_electrons)
print('C',C)
print('rho', rho)
    # One- and Two-body integrals
integrals.calculate(integrals_file)

    # Normalization of wave functions
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(bs.ALPHA[p]+bs.ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
print('S',S)

C [[0.0909769  0.64323823 0.82095787 0.14786131]
 [0.1731025  0.4450871  0.1889326  0.38876416]
 [0.85189933 0.66573975 0.56117649 0.6952178 ]
 [0.67624715 0.87305928 0.10582277 0.25480443]]
rho [[0.01655359 0.03149666 0.15500632 0.12304574]
 [0.03149666 0.05992895 0.2949318  0.23412014]
 [0.15500632 0.2949318  1.45146495 1.15218899]
 [0.12304574 0.23412014 1.15218899 0.91462041]]


C:\Programs\Anaconda3\lib\site-packages\numpy\core\_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


S [[1.20975063e+01 2.91187719e+00 3.71330014e-01 2.30637530e-02]
 [2.91187719e+00 1.42134692e+00 2.99025043e-01 2.22459699e-02]
 [3.71330014e-01 2.99025043e-01 1.41564997e-01 1.89120383e-02]
 [2.30637530e-02 2.22459699e-02 1.89120383e-02 8.24921040e-03]]


Finally, we run the self consistent field iteration to recursively update C until a convergence condition is fulfilled.

In [5]:
# Self Consistent Field
n_iterations = 0
rho_old = np.zeros((N_basis, N_basis))

while n_iterations < max_iter_SCF:
    n_iterations += 1

    F = lib.create_F_matrix(rho, integrals)
    print(F)

    if normalized_wf:
        E, C = np.linalg.eigh(F)
    else:
        F_prime = np.conjugate(X.transpose()) @ F @ X
        E, C_prime = np.linalg.eigh(F_prime)
        C = X @ C_prime

    rho = lib.density_matrix(C, N_electrons)
    total_E = lib.total_energy(rho, F, integrals)

    if lib.delta_rho(rho, rho_old) < eps_SCF:
        break

    total_E_old = total_E
    rho_old = rho

    print("E = {:0.7f} | N(SCF) = {}".format(total_E, n_iterations))

[[nan inf inf nan]
 [inf nan inf nan]
 [inf inf nan nan]
 [nan nan nan nan]]


C:\Users\alvar\Documents\TUDelft\2. Computational Physics\Project3_abermejillo_dbedialaunetar_mserraperalta\lib.py:237: RuntimeWarning: invalid value encountered in double_scalars
  F[p, q] += rho[r,s]*(integrals.get_2(p+1, q+1, r+1, s+1) - 0.5*integrals.get_2(p+1, r+1, q+1, s+1))
C:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in matmul
  


LinAlgError: Eigenvalues did not converge

The next is just a check block in which one of the two body integrals is computed. We find that its divergent :(

In [ ]:
from scipy import integrate
import numpy as np
import basis_set as bs
indices = np.array([2,2,1,1])
I = integrate.nquad(bs.He_two_body_integrand,[[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf],[-np.inf,np.inf]],args=indices,opts={"limit":100})